In [1]:
#import matplotlib
#matplotlib.use('nbAgg')

In [3]:
#!/usr/bin/env python
##############################################################################
#
# xpdacq            by Billinge Group
#                   Simon J. L. Billinge sb2896@columbia.edu
#                   (c) 2016 trustees of Columbia University in the City of
#                        New York.
#                   All rights reserved
#
# File coded by:    Timothy Liu
#
# See AUTHORS.txt for a list of people who contributed.
# See LICENSE.txt for license information.
#
##############################################################################
import os

from xpdconf.conf import glbl_dict
_exp_db = glbl_dict['exp_db']

from xpdacq.xpdacq_conf import (glbl_dict, configure_device,
                                _reload_glbl, _set_glbl,
                                _load_beamline_config)

os.makedirs(glbl_dict['home_dir'],exist_ok=True)
if not os.path.exists(glbl_dict['blconfig_path']):
     import yaml
     os.makedirs(os.path.dirname(glbl_dict['blconfig_path']), exist_ok=True)
     with open(glbl_dict['blconfig_path'], 'w') as f:
         yaml.dump({'hello': 'world'}, f)

# configure experiment device being used in current version
if glbl_dict['is_simulation']:
    from xpdacq.simulation import xpd_pe1c, cs700, shctl1, fb
    from xpdsim import *
    from ophyd.sim import hw
    hw = hw()
    from databroker import Broker
    from ophyd.sim import NumpySeqHandler
    import copy
    db = glbl_dict.get('exp_db', _exp_db)
    # db = Broker.named(glbl_dict['exp_broker_name'])
    # db.prepare_hook = lambda name, doc: copy.deepcopy(doc)
    # db.reg.register_handler('NPY_SEQ', NumpySeqHandler)

    configure_device(area_det=xpd_pe1c, shutter=shctl1,
                     temp_controller=cs700, db=db, filter_bank=fb)
else:
    # FIXME: create synthetic ring current object in fullness of time
    from ophyd import EpicsSignalRO

    ring_current = EpicsSignalRO('SR:OPS-BI{DCCT:1}I:Real-I',
                                 name='ring_current')
    configure_device(area_det=pe1c, shutter=shctl1,
                     temp_controller=cs700, db=db,
                     ring_current=ring_current)

# cache previous glbl state
reload_glbl_dict = _reload_glbl()
from xpdacq.glbl import glbl

# reload beamtime
from xpdacq.beamtimeSetup import (start_xpdacq, _start_beamtime,
                                  _end_beamtime)

bt = start_xpdacq()
if bt is not None:
    print("INFO: Reload beamtime objects:\n{}\n".format(bt))
if reload_glbl_dict is not None:
    _set_glbl(glbl, reload_glbl_dict)

# import necessary modules
from xpdacq.xpdacq import *
from xpdacq.beamtime import *
from xpdacq.utils import import_sample_info

# instantiate xrun without beamtime, like bluesky setup
xrun = CustomizedRunEngine(None)
xrun.md['beamline_id'] = glbl['beamline_id']
xrun.md['group'] = glbl['group']
xrun.md['facility'] = glbl['facility']
#beamline_config = _load_beamline_config(glbl['blconfig_path'])
#xrun.md['beamline_config'] = beamline_config

# insert header to db, either simulated or real
xrun.subscribe_lossless(db.insert, 'all')

if bt:
    xrun.beamtime = bt

HOME_DIR = glbl['home']
BASE_DIR = glbl['base']

print('INFO: Initializing the XPD data acquisition environment\n')
if os.path.isdir(HOME_DIR):
    os.chdir(HOME_DIR)
else:
    os.chdir(BASE_DIR)

from xpdacq.calib import *

# analysis functions, only at beamline
#from xpdan.data_reduction import *

#from xpdan.pipelines.callback import MainCallback
#s = MainCallback(db, glbl['tiff_base'], calibration_md_folder=glbl['config_base'])
#xrun.subscribe(s)

print('OK, ready to go.  To continue, follow the steps in the xpdAcq')
print('documentation at http://xpdacq.github.io/xpdacq\n')


No config file could be found in the following locations:
/home/mt/.config/acq
/home/mt/software/miniconda3/envs/xpd-env/etc/acq
/etc/acq
Loading from packaged simulation configuration
INFO: Reload beamtime objects:

ScanPlans:
0: ct_5
1: ct_0.1
2: ct_1
3: ct_10
4: ct_30
5: ct_60

Samples:
0: Setup
1: Ni
2: Ceo2



INFO: area detector has been configured to new acquisition time (time per frame)  = 0.1s
INFO: beamtime object has been linked

INFO: Initializing the XPD data acquisition environment

OK, ready to go.  To continue, follow the steps in the xpdAcq
documentation at http://xpdacq.github.io/xpdacq



In [5]:
#from xpdan.pipelines.main import conf_main_pipeline
#from xpdan.pipelines.callback import MainCallback
#from shed.event_streams import istar
from bluesky.utils import install_qt_kicker
import shutil
import os
from pprint import pprint
from xpdsim import xpd_wavelength
from xpdan.vend.callbacks.zmq import Proxy, Publisher, RemoteDispatcher
import multiprocessing

'''
def start_proxy():
    Proxy(5567, 5568).start()

proxy_proc = multiprocessing.Process(target=start_proxy, daemon=True)
proxy_proc.start()
time.sleep(5)  # Give this plenty of time to start up.

'''
p = Publisher('127.0.0.1:5567', RE=xrun, prefix=b'raw')  # noqa

'''
def make_and_start_dispatcher(db):
    s = MainCallback(db, glbl['tiff_base'], calibration_md_folder='~/xpdUser/config_base')
    d = RemoteDispatcher('127.0.0.1:5568')
    install_qt_kicker(loop=d.loop)
    d.subscribe(s)
    print("REMOTE IS READY TO START")
    d.start()

dispatcher_proc = multiprocessing.Process(target=make_and_start_dispatcher,
                                              daemon=True, args=(db,))

dispatcher_proc.start()
'''
#time.sleep(5)  # As above, give this plenty of time to start.
#'''
# s = conf_main_pipeline(db, glbl['tiff_base'], vis=False, write_to_disk=True, calibration_md_folder='~/xpdUser/config_base')
#pt = xrun.subscribe(istar(s.emit))
#s = MainCallback(db, glbl['tiff_base'], calibration_md_folder='~/xpdUser/config_base')
#pt = xrun.subscribe(s)
# xrun.subscribe(lambda x, y: pprint(y))
install_qt_kicker(loop=xrun.loop)
# start a beamtime
PI_name = 'Billinge '
saf_num = 30000
wavelength = xpd_wavelength
experimenters = [('van der Banerjee', 'S0ham', 1),
                 ('Terban ', ' Max', 2)]
try:
    bt = _start_beamtime(PI_name, saf_num,
                         experimenters,
                         wavelength=wavelength)
except FileExistsError:
    pass
# spreadsheet
glbl['dk_window'] = 0.1
pytest_dir = rs_fn('xpdacq', 'tests/')
xlf = '300000_sample.xlsx'
src = os.path.join(pytest_dir, xlf)
shutil.copyfile(src, os.path.join(glbl_dict['import_dir'], xlf))
import_sample_info(saf_num, bt)
xrun.beamtime = bt
# xrun(0, 1, composition_string='Ni')
# run_calibration()


INFO: using SAF_number = 30000
INFO: If you want to associate a background sample, e.g., empty kapton tube, with samples,
place the sample-name of the background sample in the column ['sample-name_of_sample_background']
of the sample excel spreadsheet.
The following samples do not have background_samples associated with them
(typically background samples won't have associated background samples):
['Setup', 'Ni', 'Ceo2']

*** End of import Sample object ***
INFO: beamtime object has been linked



In [6]:
bt.list()


ScanPlans:
0: ct_5
1: ct_0.1
2: ct_1
3: ct_10
4: ct_30
5: ct_60

Samples:
0: Setup
1: Ni
2: Ceo2


In [8]:
xrun(10,0)

INFO: requested exposure time = 5 - > computed exposure time= 5.0
[E 22:27:02.038 run_engine:1261] Run aborted
    Traceback (most recent call last):
      File "/home/mt/software/miniconda3/envs/xpd-env/lib/python3.6/site-packages/bluesky/run_engine.py", line 1150, in _run
        msg = self._plan_stack[-1].send(resp)
      File "/home/mt/software/miniconda3/envs/xpd-env/lib/python3.6/site-packages/bluesky/preprocessors.py", line 236, in msg_mutator
        msg = plan.send(ret)
      File "/home/mt/software/miniconda3/envs/xpd-env/lib/python3.6/site-packages/bluesky/preprocessors.py", line 236, in msg_mutator
        msg = plan.send(ret)
      File "/home/mt/software/miniconda3/envs/xpd-env/lib/python3.6/site-packages/bluesky/preprocessors.py", line 236, in msg_mutator
        msg = plan.send(ret)
      [Previous line repeated 1 more time]
      File "/home/mt/software/miniconda3/envs/xpd-env/lib/python3.6/site-packages/bluesky/preprocessors.py", line 500, in finalize_wrapper
        

ERROR:bluesky.RE.139728484664600:Run aborted
Traceback (most recent call last):
  File "/home/mt/software/miniconda3/envs/xpd-env/lib/python3.6/site-packages/bluesky/run_engine.py", line 1150, in _run
    msg = self._plan_stack[-1].send(resp)
  File "/home/mt/software/miniconda3/envs/xpd-env/lib/python3.6/site-packages/bluesky/preprocessors.py", line 236, in msg_mutator
    msg = plan.send(ret)
  File "/home/mt/software/miniconda3/envs/xpd-env/lib/python3.6/site-packages/bluesky/preprocessors.py", line 236, in msg_mutator
    msg = plan.send(ret)
  File "/home/mt/software/miniconda3/envs/xpd-env/lib/python3.6/site-packages/bluesky/preprocessors.py", line 236, in msg_mutator
    msg = plan.send(ret)
  [Previous line repeated 1 more time]
  File "/home/mt/software/miniconda3/envs/xpd-env/lib/python3.6/site-packages/bluesky/preprocessors.py", line 500, in finalize_wrapper
    ret = yield from plan
  File "/home/mt/software/miniconda3/envs/xpd-env/lib/python3.6/site-packages/bluesky/prepro

TypeError: 'NoneType' object is not iterable

In [7]:
xrun(10,0)

INFO: requested exposure time = 5 - > computed exposure time= 5.0
[E 22:18:00.987 run_engine:1261] Run aborted
    Traceback (most recent call last):
      File "/home/mt/software/miniconda3/envs/xpd-env/lib/python3.6/site-packages/bluesky/run_engine.py", line 1150, in _run
        msg = self._plan_stack[-1].send(resp)
      File "/home/mt/software/miniconda3/envs/xpd-env/lib/python3.6/site-packages/bluesky/preprocessors.py", line 236, in msg_mutator
        msg = plan.send(ret)
      File "/home/mt/software/miniconda3/envs/xpd-env/lib/python3.6/site-packages/bluesky/preprocessors.py", line 236, in msg_mutator
        msg = plan.send(ret)
      File "/home/mt/software/miniconda3/envs/xpd-env/lib/python3.6/site-packages/bluesky/preprocessors.py", line 236, in msg_mutator
        msg = plan.send(ret)
      [Previous line repeated 1 more time]
      File "/home/mt/software/miniconda3/envs/xpd-env/lib/python3.6/site-packages/bluesky/preprocessors.py", line 500, in finalize_wrapper
        

ERROR:bluesky.RE.140160371197936:Run aborted
Traceback (most recent call last):
  File "/home/mt/software/miniconda3/envs/xpd-env/lib/python3.6/site-packages/bluesky/run_engine.py", line 1150, in _run
    msg = self._plan_stack[-1].send(resp)
  File "/home/mt/software/miniconda3/envs/xpd-env/lib/python3.6/site-packages/bluesky/preprocessors.py", line 236, in msg_mutator
    msg = plan.send(ret)
  File "/home/mt/software/miniconda3/envs/xpd-env/lib/python3.6/site-packages/bluesky/preprocessors.py", line 236, in msg_mutator
    msg = plan.send(ret)
  File "/home/mt/software/miniconda3/envs/xpd-env/lib/python3.6/site-packages/bluesky/preprocessors.py", line 236, in msg_mutator
    msg = plan.send(ret)
  [Previous line repeated 1 more time]
  File "/home/mt/software/miniconda3/envs/xpd-env/lib/python3.6/site-packages/bluesky/preprocessors.py", line 500, in finalize_wrapper
    ret = yield from plan
  File "/home/mt/software/miniconda3/envs/xpd-env/lib/python3.6/site-packages/bluesky/prepro

TypeError: 'NoneType' object is not iterable

In [8]:
xrun?

Signature:     
xrun(
    ['sample', 'plan', 'subs=None', '*', 'verify_write=False', 'dark_strategy=<function periodic_dark at 0x7f79a0091bf8>', 'robot=False', '**metadata_kw'],
)
Type:           CustomizedRunEngine
String form:    <xpdacq.xpdacq.CustomizedRunEngine object at 0x7f79a12307f0>
File:           ~/software/miniconda3/envs/xpd-env/lib/python3.6/site-packages/xpdacq/xpdacq.py
Docstring:     
A RunEngine customized for XPD workflows.

Parameters
----------
beamtime : xpdacq.beamtime.Beamtime or None
    beamtime object that will be linked to. This beamtime object
    provide reference of sample and scanplan indicies. If no
    beamtime object is linked, index-based syntax will not be allowed.

Attributes
----------
beamtime
    beamtime object currently associated with this RunEngine instance.

Examples
--------
Basic usage: run samples and plans by number

>>> xrun(0, 0)

Advanced usage: use custom plan which is compatible with bluesky

>>> xrun(3, custom_plan)  # sample 3, a